In [1]:
import numpy as np
import pandas as pd
import os

#os.chdir('..')
os.listdir()

['preprocess_int_consumption.ipynb',
 'preprocess_imports_use.ipynb',
 'preprocess_effects.ipynb',
 'combine_import_domestic.ipynb',
 'combine_import_domestic_pi.ipynb',
 'preprocess_domestic_pxp.ipynb',
 'preprocess_import_pi.ipynb']

In [2]:
data = pd.read_excel('data/nasu1719pr.xlsx',sheet_name='IOT',header=4)
data = data.iloc[1:105,1:106]
data.tail()

FileNotFoundError: [Errno 2] No such file or directory: 'data/nasu1719pr.xlsx'

In [ ]:
cpa = pd.read_excel('data/nasu1719pr.xlsx',sheet_name='Use BP PxI',header=4)
cpa = pd.DataFrame(cpa.iloc[1:105,0])
cpa = cpa.rename(columns={cpa.columns[0]: 'CPA'})
cpa = cpa['CPA'].apply(lambda x: str(x)[4:])
cpa = pd.DataFrame(cpa)
cpa

,CPA
1,A01
2,A02
3,A03
4,B05
5,B06 & B07
...,...
100,R92
101,R93
102,S94
103,S95


In [ ]:
cpa2 = pd.read_excel('data/nasu1719pr.xlsx',sheet_name='Use BP PxI',header=3)
cpa2 = pd.DataFrame(cpa2.iloc[1:106,0])
cpa2 = cpa2.rename(columns={cpa2.columns[0]: 'CPA'})
cpa2 = cpa2['CPA'].apply(lambda x: str(x)[4:])
cpa2 = pd.DataFrame(cpa2)
cpa2

,CPA
1,
2,A01
3,A02
4,A03
5,B05
...,...
101,R92
102,R93
103,S94
104,S95


In [ ]:
data['Product'] = cpa['CPA'].astype(str).str.cat(data['Product'].astype(str), sep=' ')
data = data.T
data.index = cpa2['CPA'].astype(str).str.cat(data.index.astype(str), sep=' ')
data = data.T
data = data.rename(columns={data.columns[0]: 'Product'})
data

CPA,Product,"A01 Products of agriculture, hunting and related services","A02 Products of forestry, logging and related services",A03 Fish and other fishing products; aquaculture products; support services to fishing,B05 Coal and lignite,B06 & B07 Extraction Of Crude Petroleum And Natural Gas & Mining Of Metal Ores,B08 Other mining and quarrying products,B09 Mining support services,C101 Preserved meat and meat products,"C102_3 Processed and preserved fish, crustaceans, molluscs, fruit and vegetables",...,P85 Education services,Q86 Human health services,Q87 & Q88 Residential Care & Social Work Activities,"R90 Creative, arts and entertainment services","R91 Libraries, archives, museums and other cultural services",R92 Gambling and betting services,R93 Sports services and amusement and recreation services,S94 Services furnished by membership organisations,S95 Repair services of computers and personal and household goods,S96 Other personal services
1,"A01 Products of agriculture, hunting and relat...",3197.593647,0.0,0.0,0.0,0.000839,5.009484,0.0,5006.529214,1237.853409,...,1.681174,13.773197,32.018678,0.0,0.0,20.468691,1.921342,0.0,1.718095,0.0
2,"A02 Products of forestry, logging and related ...",114.42863,308.899083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.87056,0.0,0.939577,0.0,0.0,0.0,20.435729,0.0,1.718095,0.0
3,A03 Fish and other fishing products; aquacultu...,0.0,0.0,35.386352,0.0,0.0,0.0,0.0,6.040629,846.915095,...,0.018984,0.0,0.0,0.0,0.0,0.0,0.41765,0.0,0.0,0.0
4,B05 Coal and lignite,0.0,0.0,0.0,12.421466,0.0,3.775442,0.0,0.080184,0.018251,...,3.687642,0.0,0.0,0.0,0.0,0.0,0.0,18.360187,0.0,0.000202
5,B06 & B07 Extraction Of Crude Petroleum And Na...,0.0,0.0,0.0,0.0,729.117056,9.303328,0.002291,0.0,0.0,...,0.0,0.0,0.0,0.005019,0.0,0.067292,3.233014,0.0,0.859048,0.001487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,R92 Gambling and betting services,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00251,0.0,918.454899,1.616507,0.0,0.0,0.000744
101,R93 Sports services and amusement and recreati...,0.0,0.0,0.0,0.0,0.016137,0.0,0.0,0.000022,0.007968,...,52.548902,0.0,32.999133,13.630452,28.444294,41.398336,193.1726,5.797954,0.0,0.097452
102,S94 Services furnished by membership organisat...,0.006425,2.504587,0.0,0.0,0.032274,0.0,0.0,2.076484,0.054633,...,212.998607,0.033044,117.445429,34.33882,11.295134,13.252386,38.961308,479.297513,0.000853,0.04573
103,S95 Repair services of computers and personal ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.096841,465.339369,117.43809,0.02635,7.68632,0.353289,16.973325,0.0,200.158095,0.953119


Examine imports use table

In [ ]:
# convert wide table to long table
data_long = pd.melt(data, 
                     id_vars='Product', 
                     value_vars=data.columns.to_list()[1:]).dropna()

data_long.shape
data_long

,Product,CPA,value
0,"A01 Products of agriculture, hunting and relat...","A01 Products of agriculture, hunting and relat...",3197.593647
1,"A02 Products of forestry, logging and related ...","A01 Products of agriculture, hunting and relat...",114.42863
2,A03 Fish and other fishing products; aquacultu...,"A01 Products of agriculture, hunting and relat...",0.0
3,B05 Coal and lignite,"A01 Products of agriculture, hunting and relat...",0.0
4,B06 & B07 Extraction Of Crude Petroleum And Na...,"A01 Products of agriculture, hunting and relat...",0.0
...,...,...,...
10811,R92 Gambling and betting services,S96 Other personal services,0.000744
10812,R93 Sports services and amusement and recreati...,S96 Other personal services,0.097452
10813,S94 Services furnished by membership organisat...,S96 Other personal services,0.04573
10814,S95 Repair services of computers and personal ...,S96 Other personal services,0.953119


In [ ]:
# Strip whitespace from start and finish

def space_stripper(x):
    return x.strip()

data_long[['Product','CPA']] = data_long[['Product','CPA']].applymap(lambda x: space_stripper(x))

In [ ]:
data_long.rename(columns={'Product':'domestic input requirements'}, inplace=True)
data_long.rename(columns={'CPA':'output product'}, inplace=True)


In [ ]:
data_long['proportion'] = data_long['value'] / data_long.groupby('output product')['value'].transform('sum')

data_long.to_csv('data/iot_cleaned.csv')

In [ ]:
data_long.head()

,domestic input requirements,output product,value,proportion
0,"A01 Products of agriculture, hunting and relat...","A01 Products of agriculture, hunting and relat...",3197.593647,0.211186
1,"A02 Products of forestry, logging and related ...","A01 Products of agriculture, hunting and relat...",114.42863,0.007557
2,A03 Fish and other fishing products; aquacultu...,"A01 Products of agriculture, hunting and relat...",0.0,0.0
3,B05 Coal and lignite,"A01 Products of agriculture, hunting and relat...",0.0,0.0
4,B06 & B07 Extraction Of Crude Petroleum And Na...,"A01 Products of agriculture, hunting and relat...",0.0,0.0
